In [ ]:
#compare results from llm and from cnn
import numpy as np
import pandas as pd
import json

In [ ]:
promptOne = pd.read_csv(r'C:\Users\hagraham\OneDrive\Northwestern\458GeneralAI\Project\Results\LLMResults\api_responses.csv')
prompt2 = pd.read_csv(r'C:\Users\hagraham\OneDrive\Northwestern\458GeneralAI\Project\Results\LLMResults\second_prompt.csv')

cnnResults = pd.read_csv(r'C:\Users\hagraham\OneDrive\Northwestern\458GeneralAI\Project\Results\CNNresults.csv')

#labels
labes = pd.read_csv(r'C:\Users\hagraham\OneDrive\Northwestern\458GeneralAI\Project\labels.csv')

In [ ]:
promptOne_filtered = promptOne.iloc[:, :2]
print(promptOne_filtered)

In [ ]:
import json

def extract_breed_name(response):
    try:
        response_json = json.loads(response.replace('```json\n', '').replace('\n```', ''))
        return response_json.get('BreedName', None)
    except json.JSONDecodeError:
        return None

promptOne_filtered['breed Name'] = promptOne_filtered['response'].apply(extract_breed_name)
print(promptOne_filtered[['filename', 'breed Name']])

In [ ]:
prompt2['breed Name'] = prompt2['response'].apply(extract_breed_name)

In [ ]:
prompt2 = prompt2[['filename', 'breed Name']]

In [ ]:
prompt2 = prompt2.rename(columns={'breed Name': 'prompt2Breed'})

In [ ]:
prompt1 = promptOne_filtered[['filename', 'breed Name']]

In [ ]:
prompt1 = prompt1.rename(columns={'breed Name': 'prompt1Breed'})

In [ ]:
cnnResults = cnnResults.rename(columns={'id': 'filename', 'predicted_class_name': 'cnnBreed'})

In [ ]:
cnnResults = cnnResults[['filename', 'cnnBreed']]

In [ ]:
merged_results = prompt1.merge(prompt2, on='filename').merge(cnnResults, on='filename')
print(merged_results)

In [ ]:
# Drop the .jpg extension from the filename column
merged_results['filename'] = merged_results['filename'].str.replace('.jpg', '')

# Merge with labels dataframe on the id column
final_merged_results = merged_results.merge(labes, left_on='filename', right_on='id')
print(final_merged_results)

In [ ]:
final_merged_results['prompt1Breed'] = final_merged_results['prompt1Breed'].str.lower().str.replace('_', ' ').str.replace('-', ' ')
final_merged_results['prompt2Breed'] = final_merged_results['prompt2Breed'].str.lower().str.replace('_', ' ').str.replace('-', ' ')
final_merged_results['cnnBreed'] = final_merged_results['cnnBreed'].str.lower().str.replace('_', ' ').str.replace('-', ' ')
final_merged_results['breed'] = final_merged_results['breed'].str.lower().str.replace('_', ' ').str.replace('-', ' ')

print(final_merged_results)

In [ ]:
final_merged_results['prompt1Breed_match'] = final_merged_results['prompt1Breed'] == final_merged_results['breed']
final_merged_results['prompt2Breed_match'] = final_merged_results['prompt2Breed'] == final_merged_results['breed']
final_merged_results['cnnBreed_match'] = final_merged_results['cnnBreed'] == final_merged_results['breed']

print(final_merged_results)

In [ ]:
final_merged_results = final_merged_results[['filename', 'prompt1Breed', 'prompt1Breed_match', 'prompt2Breed', 'prompt2Breed_match', 'cnnBreed', 'cnnBreed_match', 'id', 'breed']]
print(final_merged_results)

In [ ]:
final_merged_results.to_csv('final_merged_results.csv', index=False)